Import of libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler # To standardize the data
import cvxpy as cp
from sklearn import linear_model

Import of the HMLasso function

In [2]:
### Adapt the path "C:/Users/Kilian/Desktop/ENSAE/STATAPP" to run the cell

import sys
sys.path.insert(1, 'C:/Users/Kilian/Desktop/ENSAE/STATAPP/Projet_Statapp/pretreatment')

import file_04_HMLasso as hml

## Data downloading and separation of the dataset

Dataset containing the types of each column from data_03.csv

In [3]:
columns_types = pd.read_csv("data_03_columns_types.csv", index_col=0)
columns_types.head(3)

,Name,Type
0,HHIDPN,Cont
1,HHID,Char
2,PN,Char


Downloading the data with social and genetic variables.

In [4]:
data = pd.read_csv("data_03.csv")

C:\Users\Kilian\AppData\Local\Temp\ipykernel_11068\3952530018.py:1: DtypeWarning: Columns (2684) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data_03.csv")


The column "genetic_Section_A_or_E" have mixed types, so we change its format.

In [5]:
temporary = np.where(data['genetic_Section_A_or_E'] == 'E', 1, np.where(data['genetic_Section_A_or_E'] == 'A', 0, np.nan))

In [6]:
data["genetic_Section_A_or_E"] = temporary

Now we add the health index created by t-SNE

In [7]:
tSNE_GHI = pd.read_csv("data_tSNE_GHI.csv")

We merge the t-SNE health index to the data

In [8]:
data = data.merge(tSNE_GHI, how ='left', on ='HHIDPN')

The final outcome to predict is tSNE_GHI14, so we only keep individuals who were interviewed during the last wave (14th wave)

In [9]:
data_bis = data[data['tSNE_GHI14'].notna()]

Number of individuals present in every waves.

In [10]:
tSNE_GHI[~tSNE_GHI.isnull().any(axis=1)].shape[0]

3396

We select the outcome tSNE_GHI

In [11]:
Y = data_bis[["HHIDPN"]+["tSNE_GHI" + str(i) for i in range (1,15)]]

We drop the previous health index GHIw from the data, which won't be used as outcome.
(list_columns_GHI contains the names of GHIw columns).

We drop the outcome to create the matrix X.

In [12]:
X = data_bis.drop(["GHI" + str(i) for i in range (1,15)], axis = 1)
X.drop(["tSNE_GHI" + str(i) for i in range (1,15)], axis = 1, inplace =True)

Now we split the dataset into training, validation and test sets.

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=18)
X_test, X_valid, Y_test, Y_valid = train_test_split(X_test, Y_test, test_size=0.5, random_state = 6)

## Machine learning

The objective here is to make a dataset where we observe if each variable exists at each wave

In [14]:
def dataset_temporal_variables (X_train,add_tSNE_GHIw):   
    temporal_variables = {}
    waves_columns = [col for col in X_train.columns if "genetic_" not in col and col[1] in "123456789"]
    for col in waves_columns:
      char = col[0] # R or H
      if col[2] in "01234":
        wave = col[1:3]
        suffix = col[3:]
      else:
        wave = col[1]
        suffix = col[2:]
      variable = char + 'w' + suffix

      if variable not in temporal_variables.keys():
        temporal_variables[variable] = np.zeros((14), dtype=bool)

      temporal_variables[variable][int(wave)-1] = True

    temporal_variables = pd.DataFrame(temporal_variables)

    # We manually add "tSNE_GHIw":
    if add_tSNE_GHIw and "tSNE_GHI1":
        temporal_variables["tSNE_GHIw"] = np.ones((14), dtype=bool)
        waves_columns += [f"tSNE_GHI{w}" for w in range(1,15)]
        
    return (temporal_variables,waves_columns)

In [15]:
# Timeless data
def timeless_variables(X_train,waves_columns):
    non_waves_columns = [col for col in X_train.columns if col not in waves_columns]
    To_remove = ["HHIDPN","PN","HHID","RAHHIDPN",'genetic_VERSION','genetic_Section_A_or_E']+["INW"+str(i+1) for i in range (14)]
    for x in To_remove:
        if x in non_waves_columns:
            non_waves_columns.remove(x)
    return non_waves_columns

We put the explaining variables by wave in a list of dataset Intemporal variables are put in each one of them

In [16]:
import random

def list_wave(X_train, reduced, gen_var):
    (temporal_variables , waves_columns) = dataset_temporal_variables(X_train,True)
    non_waves_columns = timeless_variables(X_train,waves_columns)
    
    non_waves_columns_no_genetic = [var for var in non_waves_columns if not('genetic_' in var)]
    
    #Reduce number of variables to code
    if reduced:
        temporal_variables_2 = temporal_variables.iloc[:,[i for i in range(1,15)]+[-i for i in range(1,5)]]
        non_waves_columns_2 = random.choices(non_waves_columns,k=5)
        non_waves_columns_no_genetic_2 = random.choices(non_waves_columns_no_genetic,k=5)

        liste = [] 
        for i in range(14):
            columns_wave_i = ["HHIDPN"]+[col.replace('w', str(i+1)) for col in temporal_variables_2.T[i].index[temporal_variables_2.T[i]] if col != "tSNE_GHIw"]
            #Add the intemporal variables only to the last wave, to avoid duplicated labels issues
            if i ==  13 and gen_var:
                liste.append(X_train.loc[X_train["INW"+str(i+1)] == 1, columns_wave_i + non_waves_columns_2])
            elif i == 13:
                liste.append(X_train.loc[X_train["INW"+str(i+1)] == 1, columns_wave_i + non_waves_columns_no_genetic_2])
            else:
                liste.append(X_train.loc[X_train["INW"+str(i+1)] == 1, columns_wave_i])
                
    #All the variables
    else:
        liste = []    # len = 14 
        for i in range(14):
            columns_wave_i = ["HHIDPN"]+[col.replace('w', str(i+1)) for col in temporal_variables.T[i].index[temporal_variables.T[i]] if col != "tSNE_GHIw"]
            #Intemporal variables only to the last wave, to avoid duplicated labels issues
            if i ==  13 and gen_var:
                liste.append(X_train.loc[X_train["INW"+str(i+1)] == 1, columns_wave_i + non_waves_columns])
            elif i == 13:
                liste.append(X_train.loc[X_train["INW"+str(i+1)] == 1, columns_wave_i + non_waves_columns_no_genetic])
            else:
                liste.append(X_train.loc[X_train["INW"+str(i+1)] == 1, columns_wave_i])
                
    return (liste)
    

### Lasso selection

We start to initialize with a first lasso on the first wave.

In [17]:
def initialize_Lasso(liste, Y_train, HMLasso, method, mu, limit,print_wave):
    
    if print_wave:
        print("wave",1)
    
    scaler = StandardScaler()#(with_std=False)
    hml.ERRORS_HANDLING = "ignore"
    
    #Prepare data
    
    ### change [0,14] to [0,1] if lasso before within
    Y_train1 = Y_train.iloc[:,[0,14]]
    Y_columns = Y_train1.columns
    Y_train1.dropna(axis =0, inplace =True)
    X_Y_tempo = liste[0].merge(Y_train1, how = "left", on = "HHIDPN")
    X_train1 = X_Y_tempo.drop(Y_columns, axis=1)
    Y_train1 = X_Y_tempo[Y_train1.columns[0]]
    Y_train1 = Y_train1.values
    Y_train1 = (Y_train1 - np.mean(Y_train1))/np.std(Y_train1)
    
  
    #HMLasso
    if HMLasso:
        #Standardize X_train
        X_train1 = scaler.fit_transform(X_train1)
        
        coefficients = apply_HMLasso(X_train1, Y_train1,mu)
        
        #Variables to keep
        var_to_keep = coefficients > 10**(limit)
    
    #Common Lasso
    else:
        X_train1 = Na_imputation(X_train1, method)
        
        #Standardize X_train
        X_train1 = scaler.fit_transform(X_train1)
        
        coefficients = apply_Lasso(X_train1, Y_train1, mu)
        
        #Variables top keep
        var_to_keep = coefficients != 0

    #Selection of variables
    if print_wave:
        print("Variables kept :", list(var_to_keep).count(1))
    var_to_keep = np.insert(var_to_keep,0,True)
    
    entire_data = liste[0]
    selected = entire_data[entire_data.columns[var_to_keep]]
        
    return selected

In [18]:
def apply_HMLasso(X, Y, mu):
    
    lasso = hml.HMLasso(mu)
    lasso.fit(X, Y)
    
    coefficients = np.abs(lasso.beta_opt.copy())
    
    return coefficients

def apply_Lasso(X,Y, mu):
    
    clf = linear_model.Lasso(alpha=mu)
    clf.fit(X, Y)
    
    coefficients = clf.coef_
    
    return coefficients

In [19]:
def Na_imputation(X, method):
    if method == "mean":
        return X.fillna(X.mean())

function to impute missing data created when merging by mean but without touching Na values already there before the merge.

In [20]:
def Na_management(df1, df2, index):
    
    merged = df1.merge(df2, how='outer', on = index)
    
    df1_index = df1.set_index(index)
    df2_index = df2.set_index(index)
    
    merged = merged.fillna(merged.mean())
    merged = merged.set_index(index)
    
    df1_index = df1_index.fillna("NaN")
    merged.update(df1_index)
    
    df2_index = df2_index.fillna("NaN")
    merged.update(df2_index)
    
    merged = merged.replace("NaN",np.nan)
    
    merged = merged.reset_index()
    
    return merged

Function to select variables by HMLasso

In [21]:
def Lasso_selection(X_train, Y_train, HMLasso, method, mu, limit, reduced, print_wave, gen_var):
    
    #If a columns contains only Nan values, we drop it
    empty_col = [col for col in X_train.columns if X_train[col].isnull().all()]
    if empty_col != []:
        X_train.drop(empty_col, axis=1, inplace=True)
    
    liste = list_wave(X_train, reduced, gen_var)
    
    if print_wave:
        print("Lasso selection, mu =", mu)
    
    selected = initialize_Lasso(liste, Y_train, HMLasso, method, mu, limit,print_wave)
    
    scaler = StandardScaler()#(with_std=False)
    hml.ERRORS_HANDLING = "ignore"
    
    for i in range (1,14) :
        
        if print_wave:
            print("wave",i+1)

        var_to_select = Na_management(selected, liste[i], "HHIDPN")
 
        ### change [i,14] to [0,i+1] if lasso before within
        Y_train_i = Y_train.iloc[:,[i,14]]
        Y_columns = Y_train_i.columns
        X_Y_train = var_to_select.merge(Y_train_i, how = 'left', on = "HHIDPN")

        Y_train_i = X_Y_train[Y_columns[0]]
        X_train_i = X_Y_train.drop(Y_columns, axis =1)

        Y_train_i = Y_train_i.fillna(Y_train_i.mean())
        Y_train_i = Y_train_i.values
        Y_train_i = (Y_train_i - np.mean(Y_train_i))/np.std(Y_train_i)

        
        #HMLasso
        if HMLasso:
            #Standardize X_train
            X_train_i = scaler.fit_transform(X_train_i)
        
            coefficients = apply_HMLasso(X_train_i, Y_train_i, mu)
        
            #Variables to keep
            var_to_keep = coefficients > 10**(limit)
            
        #Common Lasso
        else:
            X_train_i = Na_imputation(X_train_i, method)
        
            #Standardize X_train
            X_train_i = scaler.fit_transform(X_train_i)

            coefficients = apply_Lasso(X_train_i, Y_train_i, mu)

            #Variables top keep
            var_to_keep = coefficients != 0
            
        #Selection of variables
        if print_wave:
            print("Variables kept :", list(var_to_keep).count(1))
        var_to_keep = np.insert(var_to_keep,0,True)

        entire_data = var_to_select
        selected = entire_data[entire_data.columns[var_to_keep]] 
    
    #Return the data with selected variables, the number of variables selected and the names of columns
    return (selected, selected.columns)

### Within estimator

There two types of missing values, the "one-time" missing values when someone didn't awnser a question during the interview or so and the missing values when someone wasn't interviewed at all during a wave.


For the first type, we impute those missing values with the mean of the column (Nan).
(Possibility to work on another imputation method).

Then for the individuals who weren't interviewed during a wave, we replace the missing value with the temporal mean of the variable over time (NanNan)


Finally, we compute X_vague_ti = X_ti - temporal_mean(X_ti)

In [22]:
def creation_data_within(selected, Lasso_before_within):
    
    if not(Lasso_before_within):
        selected = selected.drop(["INW"+str(i) for i in range(1,15)], axis =1)
    #For the "one-time" missing values imputation by mean
    X_train_within = selected.fillna(selected.mean())
    ###For people who weren't interviewed
    
    # We start by adding the INWw columns to know if the individual was interviewed during the wave w
    X_train_within = X_train_within.merge(X_train[["HHIDPN"]+["INW"+str(i) for i in range(1,15)]], how ="left", on="HHIDPN")  

    #We recover the missing values for people who weren't interviewed during the wave w
    X_train_within = recover_missing(X_train_within)
    
    # Creation of the data set for within regression.
    (X_train_within, temporal_variables_within) = data_set_within(X_train_within)
    
    #Still Nan values in intemporal variables
    X_train_within = X_train_within.fillna(X_train_within.mean())
    
    return (X_train_within, temporal_variables_within)

This function return a dataset containing only variables concerned by the wave.

In [23]:
import re
import pickle

def get_wave(data, wave, non_temporal):
  """
  This function returns a smaller dataset summarizing all data for the given wave.

  Note that it also returns columns that are not relative to any wave (for instance, 'HHIDPN')
  """

  assert wave in range(1, 15)

  regex = re.compile("[0-9]+")
  if non_temporal:
        wave_columns = [col for col in data.columns if (len(regex.findall(col)) == 0 or regex.findall(col)[0] == str(wave))]
  else:
        wave_columns = [col for col in data.columns if (regex.findall(col) != [] and regex.findall(col)[0] == str(wave))]
  wave_data = data[wave_columns]

  return wave_data

Function to recover the missing values for people who weren't interviewed during the wave w

In [24]:
def recover_missing(X_train_within):

    X_train_within_index = X_train_within.set_index("HHIDPN")
    wave_1 = get_wave(X_train_within_index,1, non_temporal =False)
    wave_1.loc[wave_1["INW1"] == 0] = np.nan
    wave_1["INW1"].fillna(0)
    Tempo = wave_1

    for i in range(2,15):
        if i == 14:
            wave_i = get_wave(X_train_within_index, i, non_temporal =True)
        else:
            wave_i = get_wave(X_train_within_index, i, non_temporal =False)
        wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
        wave_i["INWw".replace('w', str(i))].fillna(0)
        Tempo = Tempo.merge(wave_i, how= "left", on = "HHIDPN")

    return Tempo.reset_index()

Get a dataframe to know which variables are in X_train_within

Function to compute (A faire en latex) X_vague_ti = X_ti - temporal_mean(X_ti) if the temporal variable is present in at least two waves.

It creates columns containing the temporal mean of a temporal variables and then replaces the Nan values (when people weren't interviewed) by this mean. Finally it creates the dataset  X_vague_ti.

In [25]:
def data_set_within (X_train_within):
    
    temporal_variables_within = dataset_temporal_variables(X_train_within, False)[0]
    
    X_within = X_train_within.copy()
    temporal_variables_within_actualised = temporal_variables_within.copy()
    i = 0
    
    for col in temporal_variables_within.columns:
        i += 1
        if i%10 == 0:
            print(i, "out of ", len(temporal_variables_within.columns))
        index_wave = temporal_variables_within.index[temporal_variables_within[col]==1].tolist()
        names_waves = [col.replace('w', str(i+1)) for i in index_wave]
        
        # Only if the temporal variable is present in at least two waves
        if len(names_waves)>1:
            # (~X_within[names_waves].isna()).sum(axis=1) = number of non missing values
            X_within[col+"_MEAN"] = X_within[names_waves].sum(axis=1)/(~X_within[names_waves].isna()).sum(axis=1)
            for x in names_waves:
                # Imputing the missing values by the temporal mean
                X_within[x].fillna(X_within[col+"_MEAN"], inplace= True)         
                #Creating the new data for within regression X_vague
                X_within[x+"_Within"] = X_within[x] - X_within[col+"_MEAN"]
                X_within.drop(x, inplace=True, axis=1)
        
        else:
            temporal_variables_within_actualised = temporal_variables_within_actualised.drop(col, axis=1)
    return (X_within, temporal_variables_within_actualised)

Now we do the same thing to Y_train but no need to impute the Nan values since the only outcome is tSNE_GHI14 (no Nan).

In [26]:
def creation_outcome_within(Y_train):

    Y_train_within = Y_train.copy()
    
    tSNE_GHI = [f"tSNE_GHI{w}" for w in range(1,15)]

    Y_train_within["tSNE_GHIw_MEAN"] = Y_train_within[tSNE_GHI].sum(axis=1)/(~Y_train_within[tSNE_GHI].isna()).sum(axis=1)
    for col in tSNE_GHI:
        Y_train_within[col+"_within"] = Y_train_within[col] - Y_train_within["tSNE_GHIw_MEAN"]
    
    return Y_train_within

We can now proceed to the regression

In [27]:
def get_X_Y_within(X_train_within, Y_train_within, temporal_variables_within, Lasso_before_within):
    
    data_regression = X_train_within.merge(Y_train_within[["HHIDPN","tSNE_GHI14_within"]], on = "HHIDPN")
    Y_regression = data_regression["tSNE_GHI14_within"]
    list_to_drop = ["HHIDPN","tSNE_GHI14_within"]
    if Lasso_before_within:
        list_to_drop = list_to_drop + [col+"_MEAN" for col in temporal_variables_within.columns]+["INW"+str(i) for i in range(1,15)]
        
    X_regression = data_regression.drop(list_to_drop,axis=1)
    
    return (X_regression, Y_regression)
    

In [28]:
from sklearn.linear_model import LinearRegression

def regression(X_regression,Y_regression):
    
    modeleReg=LinearRegression()

    modeleReg.fit(X_regression,Y_regression) 
    
    return modeleReg

In [29]:
def Within_estimates(X, Y, HMLasso, Lasso_before_within, method, mu, limit, reduced, print_wave, gen_var):
    
    if Lasso_before_within:
        (selected, names_var) = Lasso_selection(X, Y, HMLasso, method, mu, limit, reduced, print_wave, gen_var)

        (X_within, temporal_variables_within) = creation_data_within(selected,Lasso_before_within)
        Y_within = creation_outcome_within(Y_train)

        (X_regression, Y_regression) = get_X_Y_within(X_within, Y_within, temporal_variables_within, Lasso_before_within)
        
    else:
        
        (selected, names_var) = Lasso_selection(X, Y, HMLasso, method, mu, limit, reduced, print_wave, gen_var)
        
        temporal_variables_within = dataset_temporal_variables(selected, False)[0]
        #Still Nan values in intemporal variables
        selected_2 = selected.fillna(selected.mean())
        (X_regression, Y_regression) = get_X_Y_within(selected_2, Y, temporal_variables_within, Lasso_before_within)
    
    return (regression(X_regression,Y_regression), names_var, X_regression, Y_regression)

### Optimisation of parameters with validation set

In [30]:
def summary_model (X_train, Y_train, X_valid, Y_valid, HMLasso, Lasso_before_within, method, mu, limit, reduced, coef, print_wave, gen_var):
    
    (model, names_var, X_regression, Y_regression) = Within_estimates(X_train, Y_train, HMLasso, Lasso_before_within, method, mu, limit, reduced, print_wave, gen_var)
    
    if Lasso_before_within:
        #Selection of columns in the validation set
        selected_valid = X_valid[list(names_var)]

        (X_valid_within, temporal_variables_within) = creation_data_within(selected_valid, Lasso_before_within)
        Y_valid_within = creation_outcome_within(Y_valid)

        (X_valid_regression, Y_valid_regression) = get_X_Y_within(X_valid_within, Y_valid_within, temporal_variables_within, Lasso_before_within)
        
    else:
        
        selected_valid = X_valid[list(names_var)]
        #Still Nan values in intemporal variables
        selected_valid_2 = selected_valid.fillna(selected_valid.mean())
        temporal_variables_within = dataset_temporal_variables(selected_valid, False)[0]
        
        (X_valid_regression, Y_valid_regression) = get_X_Y_within(selected_valid_2, Y_valid, temporal_variables_within, Lasso_before_within)
        
    
    R_square_train = r_squared(X_regression, Y_regression, model)
    R_square_valid = r_squared(X_valid_regression, Y_valid_regression, model)
    
    intercept = model.intercept_
    coefficients = model.coef_
    
    RMSE_train = RMSE(X_regression, Y_regression, model)
    RMSE_valid = RMSE(X_valid_regression, Y_valid_regression, model)

    if coef:
        summary = {"HMLasso" : HMLasso, "mu" : mu, "Lasso_before_within" : Lasso_before_within, "R_square_train" : R_square_train, "RMSE_train" : RMSE_train, "R_square_valid" : R_square_valid, "RMSE_valid" : RMSE_valid, "variables kept": len(list(names_var)), "intercept" : intercept, "coefficients" : coefficients}
        return (summary, model, names_var)
    
    else:
        summary = {"HMLasso" : HMLasso, "mu" : mu, "Lasso_before_within" : Lasso_before_within, "R_square_train" : R_square_train, "RMSE_train" : RMSE_train, "R_square_valid" : R_square_valid, "RMSE_valid" : RMSE_valid, "variables kept": len(list(names_var))}
        return (summary, model, names_var)

In [31]:
from sklearn.metrics import r2_score

def r_squared(X, Y, model):
    Y_predict = model.predict(X)
    
    r2 = r2_score(Y, Y_predict)
    
    return r2

In [32]:
from sklearn.metrics import mean_squared_error

def RMSE(X, Y, model):
    Y_predict = model.predict(X)
    
    MSE = mean_squared_error(Y, Y_predict)
    
    return MSE

In [33]:
def multiples_models(X_train, Y_train, X_valid, Y_valid, list_mu, HMLasso, Lasso_before_within, method, limit, reduced, print_wave, gen_var):
    
    Frame = pd.DataFrame(columns = ["HMLasso", "mu", "Lasso_before_within", "R_square_train", "RMSE_train", "R_square_valid", "RMSE_valid", "variables kept"])
    list_model = []
    list_var = []
    
    for mu in list_mu:
        (summary_ML, model, names_var) = summary_model(X_train, Y_train, X_valid, Y_valid, HMLasso, Lasso_before_within, method, mu, limit, reduced, coef=False, print_wave=print_wave, gen_var=gen_var)
        Frame = Frame.append(summary_ML, ignore_index=True)
        list_model.append(model)
        list_var.append(names_var)
        
    return (Frame, list_model, list_var)

### Application on test set

In [34]:
def test_model (X_test, Y_test, model, names_var, Lasso_before_within):
    
    if Lasso_before_within:
        #Selection of columns in the validation set
        selected_test = X_test[list(names_var)]

        (X_test_within, temporal_variables_within) = creation_data_within(selected_test, Lasso_before_within)
        Y_test_within = creation_outcome_within(Y_test)

        (X_test_regression, Y_test_regression) = get_X_Y_within(X_test_within, Y_test_within, temporal_variables_within, Lasso_before_within)
    
    else:
        selected_test = X_test[list(names_var)]
        temporal_variables_within = dataset_temporal_variables(selected_test, False)[0]
        #Still Nan values in intemporal variables
        selected_test_2 = selected_test.fillna(selected_test.mean())
        (X_test_regression, Y_test_regression) = get_X_Y_within(selected_test_2, Y_test, temporal_variables_within, Lasso_before_within)
    
    R_square_test = model.score(X_test_regression,Y_test_regression, sample_weight=None)
    
    RMSE_test = RMSE(X_test_regression, Y_test_regression, model)
    
    return {"R_square_test" : R_square_test, "RMSE_test" : RMSE_test}


### Application (Within after Lasso)

We try to find the parameter mu that maximize the R² of the validation.
To do so, we apply machine learning algorithms with different mu to the training set and we evaluate the model with the validation set. Progressively, we get closer to the optimal mu.

Here, we select the raws variables and then we proceed to a within regression.

#### Estimates with genetic variables

In [36]:
list_mu = np.linspace(0.005,0.1,20)

In [56]:
(Frame_1, list_model_1, list_var_1) = multiples_models(X_train, Y_train, X_valid, Y_valid, list_mu, HMLasso =False, Lasso_before_within =True, method ="mean", limit= -14, reduced =False, print_wave =True, gen_var =True)

Lasso selection, mu = 0.005
wave 1
Variables kept : 303
wave 2
Variables kept : 476
wave 3
Variables kept : 566
wave 4
Variables kept : 635
wave 5
Variables kept : 671
wave 6
Variables kept : 694
wave 7
Variables kept : 714
wave 8
Variables kept : 678
wave 9
Variables kept : 678
wave 10
Variables kept : 665
wave 11
Variables kept : 646
wave 12
Variables kept : 625
wave 13
Variables kept : 596
wave 14
Variables kept : 642


C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\3054883700.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


Lasso selection, mu = 0.01
wave 1
Variables kept : 218
wave 2
Variables kept : 261
wave 3
Variables kept : 307
wave 4
Variables kept : 305
wave 5
Variables kept : 318
wave 6
Variables kept : 330
wave 7
Variables kept : 303
wave 8
Variables kept : 293
wave 9
Variables kept : 291
wave 10
Variables kept : 294
wave 11
Variables kept : 273
wave 12
Variables kept : 260
wave 13
Variables kept : 237
wave 14
Variables kept : 278


C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

...

C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\3054883700.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


Lasso selection, mu = 0.095
wave 1
Variables kept : 7
wave 2
Variables kept : 10
wave 3
Variables kept : 17
wave 4
Variables kept : 18
wave 5
Variables kept : 21
wave 6
Variables kept : 25
wave 7
Variables kept : 22
wave 8
Variables kept : 24
wave 9
Variables kept : 28
wave 10
Variables kept : 19
wave 11
Variables kept : 25
wave 12
Variables kept : 30
wave 13
Variables kept : 17
wave 14
Variables kept : 21


C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

Lasso selection, mu = 0.1
wave 1
Variables kept : 5
wave 2
Variables kept : 10
wave 3
Variables kept : 16
wave 4
Variables kept : 18
wave 5
Variables kept : 19
wave 6
Variables kept : 22
wave 7
Variables kept : 22
wave 8
Variables kept : 25
wave 9
Variables kept : 28
wave 10
Variables kept : 19
wave 11
Variables kept : 23
wave 12
Variables kept : 28
wave 13
Variables kept : 15
wave 14
Variables kept : 22


C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [57]:
Frame_1

,HMLasso,mu,R_square_train,RMSE_train,R_square_valid,RMSE_valid,variables kept
0,False,0.005,0.133328,1228.115806,2.867155e-02,1.386841e+03,643
1,False,0.010,0.097556,1278.806195,7.807376e-02,1.316306e+03,279
2,False,0.015,0.082429,1300.241906,-7.234605e+10,1.032941e+14,155
3,False,0.020,0.075657,1309.837885,8.323206e-02,1.308941e+03,117
4,False,0.025,0.072491,1314.324128,7.776578e-02,1.316745e+03,96
5,False,0.030,0.067830,1320.929074,7.472624e-02,1.321085e+03,77
6,False,0.035,0.063062,1327.685937,6.778115e-02,1.331001e+03,62
7,False,0.040,0.056857,1336.478802,6.527242e-02,1.334583e+03,57
8,False,0.045,0.052398,1342.796282,5.382288e-02,1.350931e+03,53
9,False,0.050,0.053125,1341.766386,5.017224e-02,1.356143e+03,50


In [61]:
list_mu = np.linspace(0.01,0.025,16)
list_mu

array([0.01 , 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017, 0.018,
       0.019, 0.02 , 0.021, 0.022, 0.023, 0.024, 0.025])

In [62]:
(Frame_2, list_model_2, list_var_2) = multiples_models(X_train, Y_train, X_valid, Y_valid, list_mu, HMLasso =False, Lasso_before_within =True, method ="mean", limit= -14, reduced =False, print_wave =True, gen_var = True)

Lasso selection, mu = 0.01
wave 1
Variables kept : 218
wave 2
Variables kept : 261
wave 3
Variables kept : 307
wave 4
Variables kept : 305
wave 5
Variables kept : 318
wave 6
Variables kept : 330
wave 7
Variables kept : 303
wave 8
Variables kept : 293
wave 9
Variables kept : 291
wave 10
Variables kept : 294
wave 11
Variables kept : 273
wave 12
Variables kept : 260
wave 13
Variables kept : 237
wave 14
Variables kept : 278


C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\3054883700.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


Lasso selection, mu = 0.011
wave 1
Variables kept : 196
wave 2
Variables kept : 239
wave 3
Variables kept : 282
wave 4
Variables kept : 270
wave 5
Variables kept : 295
wave 6
Variables kept : 297
wave 7
Variables kept : 275
wave 8
Variables kept : 267
wave 9
Variables kept : 268
wave 10
Variables kept : 254
wave 11
Variables kept : 238
wave 12
Variables kept : 223
wave 13
Variables kept : 198
wave 14
Variables kept : 234


C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

...

C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\3054883700.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


Lasso selection, mu = 0.025
wave 1
Variables kept : 77
wave 2
Variables kept : 81
wave 3
Variables kept : 105
wave 4
Variables kept : 101
wave 5
Variables kept : 122
wave 6
Variables kept : 97
wave 7
Variables kept : 78
wave 8
Variables kept : 96
wave 9
Variables kept : 86
wave 10
Variables kept : 90
wave 11
Variables kept : 74
wave 12
Variables kept : 83
wave 13
Variables kept : 72
wave 14
Variables kept : 95


C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\3054883700.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


In [63]:
Frame_2

,HMLasso,mu,R_square_train,RMSE_train,R_square_valid,RMSE_valid,variables kept
0,False,0.010,0.097556,1278.806195,7.807376e-02,1.316306e+03,279
1,False,0.011,0.092702,1285.684546,8.282748e-02,1.309519e+03,235
2,False,0.012,0.089091,1290.800597,7.825551e-02,1.316046e+03,204
3,False,0.013,0.088187,1292.081664,-1.079325e+10,1.541037e+13,193
4,False,0.014,0.083037,1299.379954,-1.114659e+10,1.591485e+13,170
5,False,0.015,0.082429,1300.241906,-7.234605e+10,1.032941e+14,155
6,False,0.016,0.080554,1302.899169,-7.872366e+10,1.123999e+14,144
7,False,0.017,0.077986,1306.536832,-7.017639e+10,1.001963e+14,135
8,False,0.018,0.077539,1307.171087,-1.041223e+01,1.629413e+04,128
9,False,0.019,0.077173,1307.688939,-1.405046e+01,2.148871e+04,124


In [69]:
list_mu = np.concatenate((np.linspace(0.01,0.012,11), np.linspace(0.019,0.021,11)))
list_mu

array([0.01  , 0.0102, 0.0104, 0.0106, 0.0108, 0.011 , 0.0112, 0.0114,
       0.0116, 0.0118, 0.012 , 0.019 , 0.0192, 0.0194, 0.0196, 0.0198,
       0.02  , 0.0202, 0.0204, 0.0206, 0.0208, 0.021 ])

In [70]:
(Frame_3, list_model_3, list_var_3) = multiples_models(X_train, Y_train, X_valid, Y_valid, list_mu, HMLasso =False, Lasso_before_within =True, method ="mean", limit= -14, reduced =False, print_wave =True, gen_var = True)

Lasso selection, mu = 0.01
wave 1
Variables kept : 218
wave 2
Variables kept : 261
wave 3
Variables kept : 307
wave 4
Variables kept : 305
wave 5
Variables kept : 318
wave 6
Variables kept : 330
wave 7
Variables kept : 303
wave 8
Variables kept : 293
wave 9
Variables kept : 291
wave 10
Variables kept : 294
wave 11
Variables kept : 273
wave 12
Variables kept : 260
wave 13
Variables kept : 237
wave 14
Variables kept : 278


C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\3054883700.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


Lasso selection, mu = 0.0102
wave 1
Variables kept : 212
wave 2
Variables kept : 255
wave 3
Variables kept : 300
wave 4
Variables kept : 302
wave 5
Variables kept : 309
wave 6
Variables kept : 325
wave 7
Variables kept : 299
wave 8
Variables kept : 287
wave 9
Variables kept : 290
wave 10
Variables kept : 288
wave 11
Variables kept : 261
wave 12
Variables kept : 253
wave 13
Variables kept : 230
wave 14
Variables kept : 268


C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

...

C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\3054883700.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


Lasso selection, mu = 0.021
wave 1
Variables kept : 98
wave 2
Variables kept : 102
wave 3
Variables kept : 131
wave 4
Variables kept : 127
wave 5
Variables kept : 138
wave 6
Variables kept : 120
wave 7
Variables kept : 103
wave 8
Variables kept : 115
wave 9
Variables kept : 109
wave 10
Variables kept : 103
wave 11
Variables kept : 96
wave 12
Variables kept : 96
wave 13
Variables kept : 84
wave 14
Variables kept : 109


C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\3054883700.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


In [71]:
Frame_3

,HMLasso,mu,R_square_train,RMSE_train,R_square_valid,RMSE_valid,variables kept
0,False,0.0100,0.097556,1278.806195,7.807376e-02,1.316306e+03,279
1,False,0.0102,0.097305,1279.160893,7.982161e-02,1.313810e+03,269
2,False,0.0104,0.096161,1280.783085,8.110576e-02,1.311977e+03,261
3,False,0.0106,0.095085,1282.307212,8.152428e-02,1.311379e+03,251
4,False,0.0108,0.093994,1283.853333,8.211663e-02,1.310533e+03,243
5,False,0.0110,0.092702,1285.684546,8.282748e-02,1.309519e+03,235
6,False,0.0112,0.091695,1287.111061,7.857878e-02,1.315585e+03,229
7,False,0.0114,0.091050,1288.024761,7.649767e-02,1.318556e+03,221
8,False,0.0116,0.090075,1289.406187,7.675672e-02,1.318186e+03,213
9,False,0.0118,0.089466,1290.270167,-1.135979e+11,1.621925e+14,208


In [72]:
list_mu = np.concatenate((np.linspace(0.0108,0.0112,11), np.linspace(0.0198,0.0202,11)))
list_mu

array([0.0108 , 0.01084, 0.01088, 0.01092, 0.01096, 0.011  , 0.01104,
       0.01108, 0.01112, 0.01116, 0.0112 , 0.0198 , 0.01984, 0.01988,
       0.01992, 0.01996, 0.02   , 0.02004, 0.02008, 0.02012, 0.02016,
       0.0202 ])

In [74]:
(Frame_4, list_model_4, list_var_4) = multiples_models(X_train, Y_train, X_valid, Y_valid, list_mu, HMLasso =False, Lasso_before_within =True, method ="mean", limit= -14, reduced =False, print_wave =True, gen_var = True)

Lasso selection, mu = 0.0108
wave 1
Variables kept : 198
wave 2
Variables kept : 240
wave 3
Variables kept : 284
wave 4
Variables kept : 277
wave 5
Variables kept : 298
wave 6
Variables kept : 306
wave 7
Variables kept : 280
wave 8
Variables kept : 270
wave 9
Variables kept : 273
wave 10
Variables kept : 264
wave 11
Variables kept : 242
wave 12
Variables kept : 228
wave 13
Variables kept : 209
wave 14
Variables kept : 242


C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\3054883700.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


Lasso selection, mu = 0.01084
wave 1
Variables kept : 197
wave 2
Variables kept : 240
wave 3
Variables kept : 282
wave 4
Variables kept : 276
wave 5
Variables kept : 297
wave 6
Variables kept : 308
wave 7
Variables kept : 281
wave 8
Variables kept : 269
wave 9
Variables kept : 271
wave 10
Variables kept : 264
wave 11
Variables kept : 242
wave 12
Variables kept : 227
wave 13
Variables kept : 208
wave 14
Variables kept : 240


C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

...

C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\3054883700.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


Lasso selection, mu = 0.0202
wave 1
Variables kept : 102
wave 2
Variables kept : 110
wave 3
Variables kept : 141
wave 4
Variables kept : 132
wave 5
Variables kept : 144
wave 6
Variables kept : 122
wave 7
Variables kept : 108
wave 8
Variables kept : 116
wave 9
Variables kept : 113
wave 10
Variables kept : 107
wave 11
Variables kept : 103
wave 12
Variables kept : 103
wave 13
Variables kept : 93
wave 14
Variables kept : 116


C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\3054883700.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


In [75]:
Frame_4

,HMLasso,mu,R_square_train,RMSE_train,R_square_valid,RMSE_valid,variables kept
0,False,0.01080,0.093994,1283.853333,0.082117,1310.533455,243
1,False,0.01084,0.093247,1284.911427,0.081866,1310.890997,241
2,False,0.01088,0.093199,1284.979462,0.082063,1310.609390,240
3,False,0.01092,0.093148,1285.051891,0.082387,1310.147937,239
4,False,0.01096,0.093142,1285.060002,0.082103,1310.553125,238
5,False,0.01100,0.092702,1285.684546,0.082827,1309.518518,235
6,False,0.01104,0.092697,1285.691428,0.082741,1309.642447,234
7,False,0.01108,0.092466,1286.018068,0.080824,1312.378395,232
8,False,0.01112,0.092416,1286.090082,0.080732,1312.509812,231
9,False,0.01116,0.091695,1287.111061,0.078579,1315.584714,229


In [76]:
Summary = test_model (X_test, Y_test, list_model_4[5], list_var_4[5], Lasso_before_within =True)

C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

In [77]:
Summary

{'R_square_test': 0.04152599556955783, 'RMSE_test': 1354.8380250335442}

Number of genetics variables selected by the Lasso regression

In [87]:
nb_genetic_1 = [x for x in list_var_4[5] if "genetic_" in x]
print(len(nb_genetic_1))
nb_genetic_1

12


['genetic_PC1_5E',
 'genetic_4_EVRSMK_TAG10',
 'genetic_4_ADHD_PGC17',
 'genetic_4_MENA_REPROGEN14',
 'genetic_4_LONG_CHARGE15',
 'genetic_4_NEBM_SOCGEN16',
 'genetic_4_MDD2_PGC18',
 'genetic_4_AI_GSCAN19',
 'genetic_4_SC_GSCAN19',
 'genetic_4_SI_GSCAN19',
 'genetic_4_PP_COGENT17',
 'genetic_4_HBA1CEA_MAGIC17']

We test the model which maximize the R² on the validation set.

In [78]:
Summary_2 = test_model (X_test, Y_test, list_model_4[13], list_var_4[13], Lasso_before_within =True)

C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

In [79]:
Summary_2

{'R_square_test': 0.052786164972496, 'RMSE_test': 1338.9213641696065}

Number of genetics variables selected by the Lasso regression

In [89]:
nb_genetic_2 = [x for x in list_var_4[13] if "genetic_" in x]
print(len(nb_genetic_2))
nb_genetic_2

3


['genetic_4_ADHD_PGC17', 'genetic_4_MDD2_PGC18', 'genetic_4_SI_GSCAN19']

#### Estimates without genetic variables

In [80]:
list_mu = np.linspace(0.005,0.1,20)
list_mu

array([0.005, 0.01 , 0.015, 0.02 , 0.025, 0.03 , 0.035, 0.04 , 0.045,
       0.05 , 0.055, 0.06 , 0.065, 0.07 , 0.075, 0.08 , 0.085, 0.09 ,
       0.095, 0.1  ])

In [63]:
(Frame_1_non_g, list_model_1_non_g, list_var_1_non_g) = multiples_models(X_train, Y_train, X_valid, Y_valid, list_mu, HMLasso =False, Lasso_before_within =True, method ="mean", limit= -14, reduced =False, print_wave =True, gen_var =False)

NameError: name 'X_train' is not defined

In [82]:
Frame_1_non_g

,HMLasso,mu,R_square_train,RMSE_train,R_square_valid,RMSE_valid,variables kept
0,False,0.005,0.128708,1234.662671,0.044399,1364.385927,609
1,False,0.010,0.095906,1281.143972,0.080170,1313.312862,263
2,False,0.015,0.081445,1301.635193,0.076666,1318.315823,150
3,False,0.020,0.075596,1309.923877,0.083102,1309.126735,115
4,False,0.025,0.072489,1314.327163,0.077885,1316.575648,95
5,False,0.030,0.067830,1320.929074,0.074726,1321.085291,77
6,False,0.035,0.063062,1327.685937,0.067781,1331.001326,62
7,False,0.040,0.056857,1336.478802,0.065272,1334.583249,57
8,False,0.045,0.052398,1342.796282,0.053823,1350.930643,53
9,False,0.050,0.053125,1341.766386,0.050172,1356.142949,50


In [135]:
list_mu = np.linspace(0.005,0.025,11)
list_mu

array([0.005, 0.007, 0.009, 0.011, 0.013, 0.015, 0.017, 0.019, 0.021,
       0.023, 0.025])

In [136]:
(Frame_2_non_g, list_model_2_non_g, list_var_2_non_g) = multiples_models(X_train, Y_train, X_valid, Y_valid, list_mu, HMLasso =False, Lasso_before_within =True, method ="mean", limit= -14, reduced =False, print_wave =True, gen_var =False)

Lasso selection, mu = 0.005
wave 1
Variables kept : 303
wave 2
Variables kept : 476
wave 3
Variables kept : 566
wave 4
Variables kept : 635
wave 5
Variables kept : 671
wave 6
Variables kept : 694
wave 7
Variables kept : 714
wave 8
Variables kept : 678
wave 9
Variables kept : 678
wave 10
Variables kept : 665
wave 11
Variables kept : 646
wave 12
Variables kept : 625
wave 13
Variables kept : 596
wave 14
Variables kept : 608


C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

10 out of  413
20 out of  413
30 out of  413
40 out of  413
50 out of  413
60 out of  413
70 out of  413
80 out of  413
90 out of  413
100 out of  413
110 out of  413
120 out of  413
130 out of  413
140 out of  413
150 out of  413
160 out of  413
170 out of  413
180 out of  413
190 out of  413
200 out of  413
210 out of  413
220 out of  413
230 out of  413
240 out of  413
250 out of  413
260 out of  413
270 out of  413
280 out of  413
290 out of  413
300 out of  413
310 out of  413
320 out of  413
330 out of  413
340 out of  413
350 out of  413
360 out of  413
370 out of  413
380 out of  413
390 out of  413
400 out of  413
410 out of  413


C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

10 out of  413
20 out of  413
30 out of  413
40 out of  413
50 out of  413
60 out of  413
70 out of  413
80 out of  413
90 out of  413
100 out of  413
110 out of  413
120 out of  413
130 out of  413
140 out of  413
150 out of  413
160 out of  413
170 out of  413
180 out of  413
190 out of  413
200 out of  413
210 out of  413
220 out of  413
230 out of  413
240 out of  413
250 out of  413
260 out of  413
270 out of  413
280 out of  413
290 out of  413
300 out of  413
310 out of  413
320 out of  413
330 out of  413
340 out of  413
350 out of  413
360 out of  413
370 out of  413
380 out of  413
390 out of  413
400 out of  413
410 out of  413


...

C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

10 out of  65
20 out of  65
30 out of  65
40 out of  65
50 out of  65
60 out of  65


C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

10 out of  65
20 out of  65
30 out of  65
40 out of  65
50 out of  65
60 out of  65


C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\3939267960.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


In [137]:
Frame_2_non_g

,HMLasso,mu,Lasso_before_within,R_square_train,RMSE_train,R_square_valid,RMSE_valid,variables kept
0,False,0.005,True,0.128708,1234.662677,-1.441433e+09,2.058045e+12,609
1,False,0.007,True,0.109462,1261.934519,-7.609601e+11,1.086482e+15,411
2,False,0.009,True,0.102032,1272.462684,6.891083e-02,1.329388e+03,298
3,False,0.011,True,0.091232,1287.767803,8.303248e-02,1.309226e+03,226
4,False,0.013,True,0.086996,1293.770540,-1.645393e+10,2.349255e+13,187
5,False,0.015,True,0.081443,1301.638224,-7.691872e+10,1.098228e+14,150
6,False,0.017,True,0.077819,1306.773447,-7.850552e+10,1.120884e+14,130
7,False,0.019,True,0.077113,1307.775040,8.105437e-02,1.312050e+03,120
8,False,0.021,True,0.073688,1312.627397,8.121855e-02,1.311816e+03,109
9,False,0.023,True,0.073277,1313.210355,8.093972e-02,1.312214e+03,103


We test the model which maximize the R² on the validation set.

In [138]:
Summary = test_model (X_test, Y_test, list_model_2_non_g[3], list_var_2_non_g[3], Lasso_before_within =True)

C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_13644\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

10 out of  169
20 out of  169
30 out of  169
40 out of  169
50 out of  169
60 out of  169
70 out of  169
80 out of  169
90 out of  169
100 out of  169
110 out of  169
120 out of  169
130 out of  169
140 out of  169
150 out of  169
160 out of  169


In [139]:
Summary

{'R_square_test': 0.048541195861122155, 'RMSE_test': 1344.9217831069982}

### Application (Within before Lasso)

We try to find the parameter mu that maximize the R² of the validation.
To do so, we apply machine learning algorithms with different mu to the training set and we evaluate the model with the validation set. Progressively, we get closer to the optimal mu.

We start by computing the within data sets : X_vague_ti = X_ti - temporal_mean(X_ti), and then we select variables with lasso regressions before performing the final regression.

In [36]:
print("train")
(X_within_train, temporal_variables_within) = creation_data_within(X_train, Lasso_before_within = False)
Y_within_train = creation_outcome_within(Y_train)
        
#Remove column containing the temporal mean
list_to_drop = [col+"_MEAN" for col in temporal_variables_within.columns]+["INW"+str(i) for i in range(1,15)]
X_train_mean = X_within_train[["HHIDPN"]+[col+"_MEAN" for col in temporal_variables_within.columns]]
X_within_train = X_within_train.drop(list_to_drop,axis=1)
list_to_keep = [f"tSNE_GHI{w}_within" for w in range(1,15)]+["HHIDPN"]
Y_within_train = Y_within_train[list_to_keep]

#Problem with "INWw" columns
X_within_train = X_within_train.merge(X_train, on="HHIDPN")


print("valid")
(X_within_valid, temporal_variables_within) = creation_data_within(X_valid, Lasso_before_within = False)
Y_within_valid = creation_outcome_within(Y_valid)
        
#Remove column containing the temporal mean
list_to_drop = [col+"_MEAN" for col in temporal_variables_within.columns]+["INW"+str(i) for i in range(1,15)]
X_valid_mean = X_within_valid[["HHIDPN"]+[col+"_MEAN" for col in temporal_variables_within.columns]]
X_within_valid = X_within_valid.drop(list_to_drop,axis=1)
list_to_keep = [f"tSNE_GHI{w}_within" for w in range(1,15)]+["HHIDPN"]
Y_within_valid = Y_within_valid[list_to_keep]
  
#Problem with "INWw" columns
X_within_valid = X_within_valid.merge(X_valid, on="HHIDPN")
    
print("test")
(X_within_test, temporal_variables_within) = creation_data_within(X_test, Lasso_before_within = False)
Y_within_test = creation_outcome_within(Y_test)
        
#Remove column containing the temporal mean
list_to_drop = [col+"_MEAN" for col in temporal_variables_within.columns]+["INW"+str(i) for i in range(1,15)]
X_test_mean = [["HHIDPN"]+[col+"_MEAN" for col in temporal_variables_within.columns]]
X_within_test = X_within_test.drop(list_to_drop,axis=1)
list_to_keep = [f"tSNE_GHI{w}_within" for w in range(1,15)]+["HHIDPN"]
Y_within_test = Y_within_test[list_to_keep]

#Problem with "INWw" columns
X_within_test = X_within_test.merge(X_test, on="HHIDPN")

train


C:\Users\Kilian\AppData\Local\Temp\ipykernel_11068\2199017923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_11068\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_11068\2199017923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

10 out of  1137
20 out of  1137
30 out of  1137
40 out of  1137
50 out of  1137
60 out of  1137
70 out of  1137
80 out of  1137
90 out of  1137
100 out of  1137
110 out of  1137
120 out of  1137
130 out of  1137
140 out of  1137
150 out of  1137
160 out of  1137
170 out of  1137
180 out of  1137
190 out of  1137
200 out of  1137
210 out of  1137
220 out of  1137
230 out of  1137
240 out of  1137
250 out of  1137
260 out of  1137
270 out of  1137
280 out of  1137
290 out of  1137
300 out of  1137
310 out of  1137
320 out of  1137
330 out of  1137
340 out of  1137
350 out of  1137
360 out of  1137
370 out of  1137
380 out of  1137
390 out of  1137


KeyboardInterrupt: 

In [40]:
X_within_train.to_csv('X_within_train.csv',sep = ',', index=False)
Y_within_train.to_csv('Y_within_train.csv',sep = ',', index=False)
X_within_valid.to_csv('X_within_valid.csv',sep = ',', index=False)
Y_within_valid.to_csv('Y_within_valid.csv',sep = ',', index=False)
X_within_test.to_csv('X_within_test.csv',sep = ',', index=False)
Y_within_test.to_csv('Y_within_test.csv',sep = ',', index=False)

In [ ]:
X_within_train = pd.read_csv('X_within_train.csv')
Y_within_train = pd.read_csv('Y_within_train.csv')
X_within_valid = pd.read_csv('X_within_valid.csv')
Y_within_valid = pd.read_csv('Y_within_valid.csv')
X_within_test = pd.read_csv('X_within_test.csv')
Y_within_test = pd.read_csv('Y_within_test.csv')

In [35]:
Y_within_test

NameError: name 'Y_within_test' is not defined

#### Estimates with genetic variables

In [73]:
list_mu = np.linspace(0.005,0.1,20)
list_mu

array([0.005, 0.01 , 0.015, 0.02 , 0.025, 0.03 , 0.035, 0.04 , 0.045,
       0.05 , 0.055, 0.06 , 0.065, 0.07 , 0.075, 0.08 , 0.085, 0.09 ,
       0.095, 0.1  ])

In [74]:
(Frame_within_1, list_model_within_1, list_var_within_1) = multiples_models(X_within_train, Y_within_train, X_within_valid, Y_within_valid, list_mu, HMLasso =False, Lasso_before_within =False, method ="mean", limit= -14, reduced =False, print_wave =True, gen_var =True)

Lasso selection, mu = 0.005
wave 1


C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\289977380.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train1.dropna(axis =0, inplace =True)


Variables kept : 480
wave 2
Variables kept : 717
wave 3
Variables kept : 830
wave 4
Variables kept : 889
wave 5
Variables kept : 915
wave 6
Variables kept : 977
wave 7
Variables kept : 1022
wave 8
Variables kept : 991
wave 9
Variables kept : 979
wave 10
Variables kept : 941
wave 11
Variables kept : 971
wave 12
Variables kept : 917
wave 13
Variables kept : 864
wave 14
Variables kept : 968


C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\3939267960.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


Lasso selection, mu = 0.01
wave 1


...

C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\3939267960.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


Lasso selection, mu = 0.1
wave 1


C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\289977380.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train1.dropna(axis =0, inplace =True)


Variables kept : 2
wave 2
Variables kept : 3
wave 3
Variables kept : 0
wave 4
Variables kept : 1
wave 5
Variables kept : 0
wave 6
Variables kept : 0
wave 7
Variables kept : 0
wave 8
Variables kept : 0
wave 9
Variables kept : 0
wave 10
Variables kept : 1
wave 11
Variables kept : 0
wave 12
Variables kept : 0
wave 13
Variables kept : 2
wave 14
Variables kept : 6


C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\3939267960.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


In [75]:
Frame_within_1

,HMLasso,mu,Lasso_before_within,R_square_train,RMSE_train,R_square_valid,RMSE_valid,variables kept
0,False,0.005,False,0.224834,1098.446659,-0.146074,1636.338891,969
1,False,0.010,False,0.162035,1187.435991,-0.082723,1545.887354,369
2,False,0.015,False,0.132166,1229.761458,-0.029513,1469.915493,169
3,False,0.020,False,0.113446,1256.289206,0.013969,1407.833530,86
4,False,0.025,False,0.103000,1271.090757,0.033529,1379.906464,59
5,False,0.030,False,0.098045,1278.113237,0.048617,1358.363761,48
6,False,0.035,False,0.093826,1284.091730,0.058247,1344.613424,36
7,False,0.040,False,0.090088,1289.388107,0.056671,1346.864372,28
8,False,0.045,False,0.087432,1293.151470,0.061006,1340.674145,23
9,False,0.050,False,0.086488,1294.489826,0.066334,1333.067458,20


In [79]:
list_mu = np.linspace(0.045,0.06,16)
list_mu

array([0.045, 0.046, 0.047, 0.048, 0.049, 0.05 , 0.051, 0.052, 0.053,
       0.054, 0.055, 0.056, 0.057, 0.058, 0.059, 0.06 ])

In [80]:
(Frame_within_2, list_model_within_2, list_var_within_2) = multiples_models(X_within_train, Y_within_train, X_within_valid, Y_within_valid, list_mu, HMLasso =False, Lasso_before_within =False, method ="mean", limit= -14, reduced =False, print_wave =True, gen_var =True)

Lasso selection, mu = 0.045
wave 1


C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\289977380.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train1.dropna(axis =0, inplace =True)


Variables kept : 29
wave 2
Variables kept : 22
wave 3
Variables kept : 21
wave 4
Variables kept : 27
wave 5
Variables kept : 20
wave 6
Variables kept : 19
wave 7
Variables kept : 11
wave 8
Variables kept : 13
wave 9
Variables kept : 8
wave 10
Variables kept : 13
wave 11
Variables kept : 6
wave 12
Variables kept : 14
wave 13
Variables kept : 18
wave 14
Variables kept : 22


C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\3939267960.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


Lasso selection, mu = 0.046
wave 1


...

C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\3939267960.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


Lasso selection, mu = 0.06
wave 1


C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\289977380.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train1.dropna(axis =0, inplace =True)


Variables kept : 12
wave 2
Variables kept : 12
wave 3
Variables kept : 8
wave 4
Variables kept : 13
wave 5
Variables kept : 9
wave 6
Variables kept : 5
wave 7
Variables kept : 2
wave 8
Variables kept : 2
wave 9
Variables kept : 2
wave 10
Variables kept : 3
wave 11
Variables kept : 0
wave 12
Variables kept : 7
wave 13
Variables kept : 14
wave 14
Variables kept : 16


C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\3939267960.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


In [81]:
Frame_within_2

,HMLasso,mu,Lasso_before_within,R_square_train,RMSE_train,R_square_valid,RMSE_valid,variables kept
0,False,0.045,False,0.087432,1293.151470,0.061006,1340.674145,23
1,False,0.046,False,0.087432,1293.151470,0.061006,1340.674145,23
2,False,0.047,False,0.087432,1293.151470,0.061006,1340.674145,23
3,False,0.048,False,0.086867,1293.952729,0.066011,1333.528017,22
4,False,0.049,False,0.086867,1293.952729,0.066011,1333.528017,22
5,False,0.050,False,0.086488,1294.489826,0.066334,1333.067458,20
6,False,0.051,False,0.085731,1295.562514,0.064910,1335.101403,19
7,False,0.052,False,0.085731,1295.562514,0.064910,1335.101403,19
8,False,0.053,False,0.086488,1294.489826,0.066334,1333.067458,21
9,False,0.054,False,0.086488,1294.489826,0.066334,1333.067458,21


In [82]:
Summary = test_model (X_within_test, Y_within_test, list_model_within_2[9], list_var_within_2[9], Lasso_before_within =False)

In [83]:
Summary

{'R_square_test': 0.050898317179604136, 'RMSE_test': 1341.5899060011666}

Number of genetics variables selected by the Lasso regression

In [88]:
nb_genetic_within_1 = [x for x in  list_var_within_2[9] if "genetic_" in x]
print(len(nb_genetic_within_1))

0


In [85]:
Summary_2 = test_model (X_within_test, Y_within_test, list_model_within_2[10], list_var_within_2[10], Lasso_before_within =False)

In [86]:
Summary_2

{'R_square_test': 0.050898317179604136, 'RMSE_test': 1341.5899060011666}

Number of genetics variables selected by the Lasso regression

In [87]:
nb_genetic_within_2 = [x for x in  list_var_within_2[10] if "genetic_" in x]
print(len(nb_genetic_within_2))

0


#### Estimates with genetic variables

In [89]:
list_mu = np.linspace(0.005,0.1,20)
list_mu

array([0.005, 0.01 , 0.015, 0.02 , 0.025, 0.03 , 0.035, 0.04 , 0.045,
       0.05 , 0.055, 0.06 , 0.065, 0.07 , 0.075, 0.08 , 0.085, 0.09 ,
       0.095, 0.1  ])

In [90]:
(Frame_within_1_non_gen, list_model_within_1_non_gen, list_var_within_1_non_gen) = multiples_models(X_within_train, Y_within_train, X_within_valid, Y_within_valid, list_mu, HMLasso =False, Lasso_before_within =False, method ="mean", limit= -14, reduced =False, print_wave =True, gen_var =False)

Lasso selection, mu = 0.005
wave 1


C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\289977380.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train1.dropna(axis =0, inplace =True)


Variables kept : 480
wave 2
Variables kept : 717
wave 3
Variables kept : 830
wave 4
Variables kept : 889
wave 5
Variables kept : 915
wave 6
Variables kept : 977
wave 7
Variables kept : 1022
wave 8
Variables kept : 991
wave 9
Variables kept : 979
wave 10
Variables kept : 941
wave 11
Variables kept : 971
wave 12
Variables kept : 917
wave 13
Variables kept : 864
wave 14
Variables kept : 904


C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\3939267960.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


Lasso selection, mu = 0.01
wave 1


...

C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\3939267960.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


Lasso selection, mu = 0.1
wave 1


C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\289977380.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train1.dropna(axis =0, inplace =True)


Variables kept : 2
wave 2
Variables kept : 3
wave 3
Variables kept : 0
wave 4
Variables kept : 1
wave 5
Variables kept : 0
wave 6
Variables kept : 0
wave 7
Variables kept : 0
wave 8
Variables kept : 0
wave 9
Variables kept : 0
wave 10
Variables kept : 1
wave 11
Variables kept : 0
wave 12
Variables kept : 0
wave 13
Variables kept : 2
wave 14
Variables kept : 6


C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\3939267960.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


In [91]:
Frame_within_1_non_gen

,HMLasso,mu,Lasso_before_within,R_square_train,RMSE_train,R_square_valid,RMSE_valid,variables kept
0,False,0.005,False,0.215444,1111.753197,-0.140131,1627.853011,905
1,False,0.010,False,0.155375,1196.872995,-0.074770,1534.533152,340
2,False,0.015,False,0.128814,1234.512173,-0.024086,1462.167235,160
3,False,0.020,False,0.111081,1259.640844,0.020305,1398.786276,80
4,False,0.025,False,0.102532,1271.754953,0.034745,1378.170153,58
5,False,0.030,False,0.098045,1278.113237,0.048617,1358.363761,48
6,False,0.035,False,0.093826,1284.091730,0.058247,1344.613424,36
7,False,0.040,False,0.090088,1289.388107,0.056671,1346.864372,28
8,False,0.045,False,0.087432,1293.151470,0.061006,1340.674145,23
9,False,0.050,False,0.086488,1294.489826,0.066334,1333.067458,20


In [92]:
list_mu = np.linspace(0.045,0.06,16)
list_mu

array([0.045, 0.046, 0.047, 0.048, 0.049, 0.05 , 0.051, 0.052, 0.053,
       0.054, 0.055, 0.056, 0.057, 0.058, 0.059, 0.06 ])

In [93]:
(Frame_within_2_non_gen, list_model_within_2_non_gen, list_var_within_2_non_gen) = multiples_models(X_within_train, Y_within_train, X_within_valid, Y_within_valid, list_mu, HMLasso =False, Lasso_before_within =False, method ="mean", limit= -14, reduced =False, print_wave =True, gen_var =False)

Lasso selection, mu = 0.045
wave 1


C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\289977380.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train1.dropna(axis =0, inplace =True)


Variables kept : 29
wave 2
Variables kept : 22
wave 3
Variables kept : 21
wave 4
Variables kept : 27
wave 5
Variables kept : 20
wave 6
Variables kept : 19
wave 7
Variables kept : 11
wave 8
Variables kept : 13
wave 9
Variables kept : 8
wave 10
Variables kept : 13
wave 11
Variables kept : 6
wave 12
Variables kept : 14
wave 13
Variables kept : 18
wave 14
Variables kept : 22


C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\3939267960.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


Lasso selection, mu = 0.046
wave 1


...

C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\3939267960.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


Lasso selection, mu = 0.06
wave 1


C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\289977380.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train1.dropna(axis =0, inplace =True)


Variables kept : 12
wave 2
Variables kept : 12
wave 3
Variables kept : 8
wave 4
Variables kept : 13
wave 5
Variables kept : 9
wave 6
Variables kept : 5
wave 7
Variables kept : 2
wave 8
Variables kept : 2
wave 9
Variables kept : 2
wave 10
Variables kept : 3
wave 11
Variables kept : 0
wave 12
Variables kept : 7
wave 13
Variables kept : 14
wave 14
Variables kept : 16


C:\Users\Kilian\AppData\Local\Temp\ipykernel_1928\3939267960.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Frame = Frame.append(summary_ML, ignore_index=True)


In [95]:
Frame_within_2_non_gen

,HMLasso,mu,Lasso_before_within,R_square_train,RMSE_train,R_square_valid,RMSE_valid,variables kept
0,False,0.045,False,0.087432,1293.151470,0.061006,1340.674145,23
1,False,0.046,False,0.087432,1293.151470,0.061006,1340.674145,23
2,False,0.047,False,0.087432,1293.151470,0.061006,1340.674145,23
3,False,0.048,False,0.086867,1293.952729,0.066011,1333.528017,22
4,False,0.049,False,0.086867,1293.952729,0.066011,1333.528017,22
5,False,0.050,False,0.086488,1294.489826,0.066334,1333.067458,20
6,False,0.051,False,0.085731,1295.562514,0.064910,1335.101403,19
7,False,0.052,False,0.085731,1295.562514,0.064910,1335.101403,19
8,False,0.053,False,0.086488,1294.489826,0.066334,1333.067458,21
9,False,0.054,False,0.086488,1294.489826,0.066334,1333.067458,21


In [98]:
Summary = test_model (X_within_test, Y_within_test, list_model_within_2_non_gen[8], list_var_within_2_non_gen[8], Lasso_before_within =False)

In [97]:
Summary

{'R_square_test': 0.050898317179604136, 'RMSE_test': 1341.5899060011666}

Same results for the algorithms with mu = 0.054 or mu = 0.055